In [1]:
import warnings
warnings.filterwarnings( 'ignore' )

In [2]:
import pandas as  pd
import numpy as np
import pickle
import joblib

In [3]:
import variables_nombres as vn
import funciones as fun

## 1. Datos del periodo 17 - 19

## 1.1. Cargar los datos

In [4]:
path_1719 = r'..\..\input\prediction\built_data\base17_19.csv'
data_1719 = pd.read_csv( path_1719 )

## 1.2. Filtrar por porcentaje de valores perdidos

In [5]:
data_1719 = fun.filtro_missings( data_1719, 0.1 )

In [6]:
data_1719.shape

(43137, 381)

## 1.3. Filtrar columnas 

In [7]:
cols_to_drop   = vn.gasto_variables + vn.gastos_ingresos_vars + vn.no_colapsables_vars + vn.string_variables + vn.no_predictoras_variables
pred_vars_1719 = [ col for col in data_1719.columns if col not in cols_to_drop ]
data_1719      = data_1719[ pred_vars_1719 ]

## 1.4. Imputar con media y con moda

In [8]:
data_1719 = fun.imputar_ii( data_1719, vn.num_vars, num = True, dummy = False )
data_1719 = fun.imputar_ii( data_1719, vn.categ_vars, num = False, dummy = False )

## 1.5. Binarizar variables multicategóricas

In [9]:
subset_binarizado        = []
variables_para_binarizar = [ var for var in vn.variables_para_binarizar if var in data_1719.columns ]
for col in variables_para_binarizar: 
    columna_binarizada   = pd.get_dummies( data_1719[ col ],
                                           prefix = col )
    subset_binarizado.append( columna_binarizada )
    
data_1719 = pd.concat( [ data_1719 ] + subset_binarizado, 
                       axis = 1 )
data_1719 = data_1719.drop( columns = variables_para_binarizar )

## 1.6. Colapsar

In [10]:
columnas_binarizadas = []
for df in subset_binarizado:
    columnas_binarizadas.extend( df.columns.tolist() )  

In [11]:
dummy_vars     = vn.variables_para_colapsar_dummies + columnas_binarizadas
dummy_vars     = [ var for var in dummy_vars if var not in vn.variables_para_binarizar and var in data_1719.columns ]
household_vars = [ var for var in vn.variables_nivel_hogar if var not in vn.no_colapsables_vars and var in data_1719.columns ]
mean_vars      = [ var for var in vn.variables_para_colapsar_prom if var in data_1719.columns ]

In [12]:
agg_dict = { **{ col: 'max' for col in dummy_vars }, 
             **{ col: 'mean' for col in mean_vars },
             **{ col: 'first' for col in household_vars } }
data_1719 = data_1719.groupby( 'numpanh' ).agg( agg_dict ).reset_index()

## 1.7. Exportar datos

In [13]:
data_1719.to_csv( r'..\..\input\prediction\preprocessed_data\base17_19_hogar.csv', 
                  index = False )

# 2. Datos del periodo 19 - 21

## 2.1. Cargar los datos

In [14]:
path_1921 = r'..\..\input\prediction\built_data\base19_21.csv'
data_1921 = pd.read_csv( path_1921 )

## 2.2. Filtrar por porcentaje de valores perdidos

In [15]:
data_1921 = fun.filtro_missings( data_1921, 0.1 )

In [16]:
data_1921.shape

(39807, 359)

## 2.3. Filtrar columnas 

In [17]:
cols_to_drop   = vn.gasto_variables + vn.gastos_ingresos_vars + vn.no_colapsables_vars + vn.string_variables + vn.no_predictoras_variables
pred_vars_1921 = [ col for col in data_1921.columns if col not in cols_to_drop ]
data_1921      = data_1921[ pred_vars_1921 ]

## 2.4. Imputar con media y con moda

In [18]:
data_1921 = fun.imputar_ii( data_1921, vn.num_vars, num = True, dummy = False )
data_1921 = fun.imputar_ii( data_1921, vn.categ_vars, num = False, dummy = False )

## 2.5. Binarizar variables multicategóricas

In [19]:
subset_binarizado_1719   = []
variables_para_binarizar = [ var for var in vn.variables_para_binarizar if var in data_1921.columns ]

for col in variables_para_binarizar: 
    columna_binarizada   = pd.get_dummies( data_1921[ col ],
                                           prefix = col )
    subset_binarizado_1719.append( columna_binarizada )
    
data_1921 = pd.concat( [ data_1921 ] + subset_binarizado_1719, 
                       axis = 1 )
data_1921 = data_1921.drop( columns = variables_para_binarizar )

## 2.6. Colapsar

In [20]:
columnas_binarizadas = []
for df in subset_binarizado_1719:
    columnas_binarizadas.extend( df.columns.tolist() )  

In [21]:
dummy_vars     = vn.variables_para_colapsar_dummies + columnas_binarizadas
dummy_vars     = [ var for var in dummy_vars if var not in vn.variables_para_binarizar and var in data_1921.columns ]
household_vars = [ var for var in vn.variables_nivel_hogar if var not in vn.no_colapsables_vars and var in data_1921.columns ]
mean_vars      = [ var for var in vn.variables_para_colapsar_prom if var in data_1921.columns ]

In [22]:
agg_dict = { **{ col: 'max' for col in dummy_vars }, 
             **{ col: 'mean' for col in mean_vars },
             **{ col: 'first' for col in household_vars } }
data_1921 = data_1921.groupby( 'numpanh' ).agg( agg_dict ).reset_index()

## 2.7. Exportar datos

In [23]:
data_1921.to_csv( r'..\..\input\prediction\preprocessed_data\base19_21_hogar.csv', 
                  index = False )